In [1]:
# import pandas library and load the datast
import pandas as pd

df=pd.read_excel(r"C:\Users\khanu\Documents\classexcelr\assignment downloads\Association Rules\Online retail.xlsx",header=None)
df

,0
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."
...,...
7496,"butter,light mayo,fresh bread"
7497,"burgers,frozen vegetables,eggs,french fries,ma..."
7498,chicken
7499,"escalope,green tea"


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7501 non-null   object
dtypes: object(1)
memory usage: 58.7+ KB


In [3]:
# summary statistics
df.describe()

,0
count,7501
unique,5176
top,cookies
freq,223


In [4]:
df.shape

(7501, 1)

In [5]:
# cheking for null values
df.isnull().sum()

0    0
dtype: int64

In [6]:
# Drop rows with missing values
df.dropna(inplace=True)

In [7]:
# Remove duplicates
df.drop_duplicates(inplace=True)

In [8]:
# split the transactions based on commas

transactions=df.iloc[:,0].apply(lambda x: x.split(','))
transactions=transactions.tolist()
print(transactions[:5])

[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs'], ['chutney'], ['turkey', 'avocado'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea']]


In [9]:
!pip install mlxtend
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [10]:
# Use the TransactionEncoder to transform the list into a one-hot encoded dataframe
te=TransactionEncoder()
te_ary=te.fit(transactions).transform(transactions)
df_tf=pd.DataFrame(te_ary,columns=te.columns_)

In [11]:
# Apply the Apriori algorithm with a minimum support threshold
frequent_items= apriori(df_tf,min_support=0.01,use_colnames=True)
frequent_items

,support,itemsets
0,0.029366,(almonds)
1,0.011206,(antioxydant juice)
2,0.045981,(avocado)
3,0.012558,(bacon)
4,0.015456,(barbecue sauce)
...,...,...
433,0.014683,"(mineral water, spaghetti, olive oil)"
434,0.016615,"(mineral water, spaghetti, pancakes)"
435,0.012365,"(mineral water, spaghetti, shrimp)"
436,0.010819,"(mineral water, soup, spaghetti)"


In [12]:
# Generate association rules with minimum confidence threshold
rules=association_rules(frequent_items,metric='confidence',min_threshold=0.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(almonds),(mineral water),0.029366,0.299845,0.011012,0.375000,1.250644,0.002207,1.120247,0.206476
1,(avocado),(chocolate),0.045981,0.205178,0.010240,0.222689,1.085347,0.000805,1.022528,0.082426
2,(avocado),(french fries),0.045981,0.192620,0.011592,0.252101,1.308800,0.002735,1.079531,0.247313
3,(avocado),(milk),0.045981,0.170015,0.010819,0.235294,1.383957,0.003002,1.085364,0.290806
4,(avocado),(mineral water),0.045981,0.299845,0.016036,0.348739,1.163064,0.002248,1.075076,0.146960
...,...,...,...,...,...,...,...,...,...,...
351,"(spaghetti, shrimp)",(mineral water),0.030332,0.299845,0.012365,0.407643,1.359511,0.003270,1.181981,0.272714
352,"(mineral water, soup)",(spaghetti),0.033423,0.229521,0.010819,0.323699,1.410327,0.003148,1.139255,0.301005
353,"(soup, spaghetti)",(mineral water),0.020672,0.299845,0.010819,0.523364,1.745448,0.004621,1.468952,0.436096
354,"(mineral water, tomatoes)",(spaghetti),0.034583,0.229521,0.013524,0.391061,1.703817,0.005587,1.265282,0.427880


In [13]:
# Confidence > 0.5: Suggests that more than 50% of the time, if the antecedent is purchased, the consequent is also purchased.
# Lift > 1: Indicates that the items are positively associated and appear together more often than random chance.

# Filter rules based on confidence and lift for more meaningful results
filtered_rules = rules[(rules['confidence'] > 0.5) & (rules['lift'] > 1.0)]

# Sort by lift to get the strongest associations
filtered_rules = filtered_rules.sort_values(by='lift', ascending=False)

# Display the top 10 rules
filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

,antecedents,consequents,support,confidence,lift
305,"(frozen vegetables, ground beef)",(spaghetti),0.012558,0.511811,2.229911
339,"(soup, milk)",(mineral water),0.012365,0.576577,1.922913
303,"(frozen vegetables, ground beef)",(mineral water),0.013331,0.543307,1.811957
353,"(soup, spaghetti)",(mineral water),0.010819,0.523364,1.745448
328,"(pancakes, ground beef)",(mineral water),0.010819,0.518519,1.729286
204,"(chicken, chocolate)",(mineral water),0.011012,0.518182,1.728163
333,"(olive oil, milk)",(mineral water),0.012365,0.512000,1.707546
252,"(olive oil, chocolate)",(mineral water),0.011978,0.508197,1.694862
323,"(milk, ground beef)",(mineral water),0.016036,0.506098,1.687861
282,"(eggs, ground beef)",(mineral water),0.014490,0.503356,1.678717


**Insights**

1. (Ground Beef, Frozen Vegetables) -> Spaghetti

Support: 1.26%
Confidence: 51.18%
Lift: 2.23
Insight: Strong association; likely meal-prep pattern for pasta dishes. Bundle these ingredients for promotions.

2. (Milk, Soup) -> Mineral Water

Support: 1.24%
Confidence: 57.66%
Lift: 1.92
Insight: Customers buying "milk" and "soup" also buy "mineral water," possibly due to health-conscious habits. Promote these together.

3. (Ground Beef, Frozen Vegetables) -> Mineral Water

Support: 1.33%
Confidence: 54.33%
Lift: 1.81
Insight: Moderate association; customers may buy these items for balanced meals. Cross-sell "mineral water" with meal ingredients.

**General Insights:**

* Focus on Meal Completeness: Customers are purchasing items that suggest they are preparing full meals, indicating a preference for convenience and complete cooking solutions.
* Target Health Trends: Insights point to a customer base that values health and hydration, providing opportunities for marketing healthy food pairings and beverages.
* Bundling and Promotions: There’s potential for creating effective promotions that bundle high-frequency items together, leveraging strong associations to increase basket sizes and customer satisfaction.

**Interview Questions:**

**What is lift and why is it important in Association rules?**

Lift measures how much more likely two items are to be bought together compared to if they were independent. A lift value > 1 means the rule is useful, as the items are bought together more often than by random chance.

**What is Support and Confidence? How do you calculate them?**

Support:
Definition: Support is the proportion of transactions in the dataset that contain a particular item or itemset.
formula:
Support (A)= Number of transactions containing A/ Total number of transactions
​
Confidence:
Definition: Confidence is the likelihood that item B is purchased given that item A has already been purchased. It reflects the conditional probability of a rule.
Formula:
Confidence (A → B)= Support (A ∩ B)/ Support (A)

**What are some limitations or challenges of Association Rule Mining?**

Association rule mining can provide valuable insights, but there are several limitations and challenges:

1. Scalability and Computational Complexity:The number of potential itemsets increases exponentially as the number of items grows, leading to high computational complexity. This can make the process slow or even infeasible for large datasets.
2. Sparsity of Data:In many real-world datasets, especially retail or e-commerce data, the data is sparse. Most customers purchase only a small subset of all possible items, leading to difficulty in finding meaningful patterns or associations.
3. Choice of Thresholds:Setting appropriate thresholds for support, confidence, and lift is challenging. If thresholds are set too high, meaningful associations may be missed. If set too low, too many insignificant rules may be generated.
4. Lack of Contextual Information:Association rules only focus on finding relationships between items without considering contextual factors like time of purchase, customer demographics, or seasonality, which could provide deeper insights.
5. Redundant or Uninteresting Rules:The Apriori algorithm may generate a large number of rules, many of which could be redundant or uninteresting. Filtering out irrelevant rules is a challenge that requires careful post-processing.
6. Interpretation of Results:While support and confidence can indicate relationships, interpreting these results in a meaningful business context can be difficult. For example, a rule may have high confidence but might not be useful in practice.
7. Overfitting:There is a risk of overfitting to the dataset, meaning the discovered rules may not generalize well to new data or future transactions, especially if the dataset is small or biased.